In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

This notebook was contributed by [Mohammad Al-Ansari](https://github.com/Mansari).

# E2E ML on GCP: MLOps stage 1 : data management: create an unlabelled Vertex AI AutoML text entity extraction dataset from PDFs using Vision API

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/datasets/get_started_with_visionapi_and_vertex_datasets.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/datasets/get_started_with_visionapi_and_vertex_datasets.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/datasets/get_started_with_visionapi_and_vertex_datasets.ipynb">
     <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview

This notebook creates an unlabelled `Vertex AI AutoML` text entity extraction dataset based on a collection of PDF files stored in a Cloud Storage bucket. 

The notebook can be modified to create different types of text datasets including sentiment analysis and classification.

### Objective

In this tutorial, you learn to use `Vision API` to extract text from PDF files stored on a Cloud Storage bucket. You then process the results and create an unlabelled `Vertex AI Dataset`, compatible with `AutoML`, for text entity extraction.

You can then either use Google Cloud console to annotate / label the dataset, or create a labelling job as demonstrated in [this notebook](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage1/get_started_with_data_labeling.ipynb).

This tutorial uses the following Google Cloud services:

- `Vision AI`
- `Vertex AI AutoML`

The steps performed include:

1. Using `Vision API` to perform Optical Character Recognition (OCR) to extract text from PDF files.
2. Processing the results and saving them to text files.
3. Generating a `Vertex AI Dataset` import file.
4. Creating a new unlabelled text entity extraction `Vertex AI Dataset` resource in `Vertex AI`.

### Dataset

The dataset used for this tutorial is the [Patent PDF Samples with Extracted Structured Data](https://console.cloud.google.com/marketplace/product/global-patents/labeled-patents) from Google Public Data Sets. 

This dataset includes data extracted from over 300 patent documents issued in the US and EU. The dataset includes links to Cloud Storage blobs for the first page of each patent, in addition to a number of extracted entities. 

The data is published as a [public dataset](https://cloud.google.com/bigquery/public-data) on `BigQuery`.

### Costs

This tutorial uses billable components of Google Cloud:

* Vision API
* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Vision API pricing](https://cloud.google.com/vision/pricing), [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

If you are using Colab or Vertex AI Workbench Notebooks, your environment already meets all the requirements to run this notebook. You can skip this step.

Otherwise, make sure your environment meets this notebook's requirements. You need the following:

- The Vision API SDK
- The Vertex AI SDK
- The Cloud Storage SDK
- Git
- Python 3
- virtualenv
- Jupyter notebook running in a virtual environment with Python 3

The Cloud Storage guide to [Setting up a Python development environment](https://cloud.google.com/python/setup) and the [Jupyter installation guide](https://jupyter.org/install) provide detailed instructions for meeting these requirements. The following steps provide a condensed set of instructions:

1. [Install and initialize the SDKs](https://cloud.google.com/sdk/docs/).

2. [Install Python 3](https://cloud.google.com/python/setup#installing_python).

3. [Install virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) and create a virtual environment that uses Python 3.  Activate the virtual environment.

4. To install Jupyter, run `pip3 install jupyter` on the command-line in a terminal shell.

5. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

6. Open this notebook in the Jupyter Notebook Dashboard.


## Installation

Install the packages required for executing this notebook. You can ignore errors for the `pip` dependecy resolver as they do not impact this notebook.

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-storage \
                         google-cloud-vision \
                         google-cloud-aiplatform \
                         google-api-core==2.10 $USER_FLAG -q

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU runtime

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the following APIs: Vision API, Vertex AI APIs, Compute Engine APIs, and Cloud Storage.](https://console.cloud.google.com/flows/enableapi?apiid=vision.googleapis.com,aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. If you are running this notebook locally, you need to install the [Cloud SDK]((https://cloud.google.com/sdk)).

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$`.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

### Regions

#### Vision AI

You can now specify continent-level data storage and Optical Character Regonition (OCR) processing by setting the `VISION_AI_REGION` variable. You can select one of the following options:

* USA country only: `us`
* The European Union: `eu`

Learn more about [Vision AI regions for OCR](https://cloud.google.com/vision/docs/pdf#regionalization)

#### Vertex AI

You can also change the `VERTEX_AI_REGION` variable, which is used for operations throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
VISION_AI_REGION = "[your-region]"  # @param {type: "string"}

if VISION_AI_REGION == "[your-region]":
    VISION_AI_REGION = "us"

VERTEX_AI_REGION = "[your-region]"  # @param {type: "string"}

if VERTEX_AI_REGION == "[your-region]":
    VERTEX_AI_REGION = "us-central1"

### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Vertex AI dataset import schema

This constant tells Vertex AI the schema for importing the dataset. In this tutorial you are going to use the value for text extraction, but you can also change it to any of the values below for other use cases:

- 
`aiplatform.schema.dataset.ioformat.text.single_label_classification`

- 
`aiplatform.schema.dataset.ioformat.text.multi_label_classification`

- 
`aiplatform.schema.dataset.ioformat.text.extraction`

- 
`aiplatform.schema.dataset.ioformat.text.sentiment`


In [ ]:
from google.cloud import aiplatform

DATASET_IMPORT_SCHEMA = aiplatform.schema.dataset.ioformat.text.extraction

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench**, your environment is already authenticated. Skip this step. If you receive errors still, you may have to grant the service account that is your Workbench notebook is running under access to the services listed below.

**If you are using Colab**, run the cell below and follow the instructions when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

In the Cloud Console, go to the [Create service account key](https://console.cloud.google.com/apis/credentials/serviceaccountkey) page.

**Click Create service account**.

In the **Service account name** field, enter a name, and click **Create**.

In the **Grant this service account access to project** section, click the Role drop-down list. Type "Vertex" into the filter box, and select **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

Click Create. A JSON file that contains your key downloads to your local environment.

Enter the path to your service account key as the GOOGLE_APPLICATION_CREDENTIALS variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex AI SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions. This bucket will be also used to store the output of the Vision API SDK PDF-to-text conversion process.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $VERTEX_AI_REGION -p $PROJECT_ID $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

### Set up variables

Next, set up some variables used throughout the tutorial.

### Import libraries and define constants

In [ ]:
import os

from google.cloud import aiplatform, storage, vision

### Initialize Vision API SDK for Python

Initialize the `Vision AI` SDK for Python for your project and region.

In [ ]:
vision_client_options = {
    "quota_project_id": PROJECT_ID,
    "api_endpoint": f"{VISION_AI_REGION}-vision.googleapis.com",
}
vision_client = vision.ImageAnnotatorClient(client_options=vision_client_options)

### Initialize Vertex AI SDK for Python

Initialize the `Vertex AI` SDK for Python for your project, region and corresponding bucket.

In [ ]:
aiplatform.init(
    project=PROJECT_ID, location=VERTEX_AI_REGION, staging_bucket=BUCKET_URI
)

### Initialize Cloud Storage SDK for Python

Initialize the `Cloud Storage` SDK for Python for your project.

In [ ]:
storage_client = storage.Client(project=PROJECT_ID)

## Tutorial

Now you are ready to start creating an unlabelled `Vertex AI Dataset` text entity extraction dataset from PDF files.

### Convert PDF files to text using Vision API

First, you make a `Vision API` request to OCR to text the PDFs from the Patent samples stored in the Cloud Storage bucket.

*Note:* `Visions API` only allows batches of 100 document submissions at a time.

In [ ]:
ORIGIN_BUCKET_NAME = "gcs-public-data--labeled-patents"
# You can add a path if needed
ORIGIN_BUCKET_PATH = ""

DESTINATION_BUCKET_NAME = BUCKET_NAME
DESTINATION_BUCKET_PATH = "ocr-output"

gcs_destination_uri = f"gs://{DESTINATION_BUCKET_NAME}/{DESTINATION_BUCKET_PATH}"

# Specify the feature for the Vision API processor
feature = vision.Feature(type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)

# Retrieve a list of all files in the bucket and path
blobs = storage_client.list_blobs(
    ORIGIN_BUCKET_NAME, prefix=ORIGIN_BUCKET_PATH, delimiter="/"
)

# Create a collection of requests. The SDK requires a separate request per each
# file that we want to extract text from
async_requests = []

# Visions API only supports processing up to 100 documents at a time
# so we will process the first 100 elements only
sliced_blob_list = list(blobs)[:100]

# Loop through the source bucket and create a request for each file there
for blob in sliced_blob_list:
    # Build input_config
    # Ensure we are only processing PDF files
    if blob.name.endswith(".pdf"):
        gcs_source = vision.GcsSource(uri=f"gs://{ORIGIN_BUCKET_NAME}/{blob.name}")
        input_config = vision.InputConfig(
            gcs_source=gcs_source, mime_type="application/pdf"
        )

        # Build output config
        # Get file name
        file_name = os.path.splitext(os.path.basename(blob.name))[0]
        gcs_destination = vision.GcsDestination(
            uri=f"{gcs_destination_uri}/{file_name}-"
        )
        output_config = vision.OutputConfig(gcs_destination=gcs_destination)

        # Build request object and add to the collection
        async_request = vision.AsyncAnnotateFileRequest(
            features=[feature], input_config=input_config, output_config=output_config
        )

        async_requests.append(async_request)

print(f"Created {len(async_requests)} requests")

# Submit the batch OCR job

operation = vision_client.async_batch_annotate_files(requests=async_requests)
print("Submitting the batch OCR job")

print("Waiting for the operation to finish... this will take a short while")

response = operation.result(timeout=420)

print("Completed!")

#### Quick peek at extracted annotated JSON files

Next, you take a peek at the contents of one of the extracted JSON annotated files.

In [ ]:
json_files = ! gsutil ls {gcs_destination_uri}

example = json_files[0]
! gsutil cat {example} | head -n 1

### Process results and build the import file

The `Vision API` output is in JSON format, and contains detailed text extraction data. You only need the full text output, so you will processs the JSON results, extract the text output, and save it in new text files to be used later in the tutorial.

In [ ]:
import json

print("Extracting text from Vision API output and saving it to text files")

ocr_blobs = storage_client.list_blobs(
    DESTINATION_BUCKET_NAME, prefix=DESTINATION_BUCKET_PATH
)

output_bucket = storage_client.bucket(DESTINATION_BUCKET_NAME)

# begin building the import file content
import_file_entries = []

for ocr_blob in ocr_blobs:
    # Only process .json files, in case we previously processed files and had .txt files
    if ocr_blob.name.endswith(".json"):
        print(f"Extracting text from {ocr_blob.name}")
        # read each blob into a stream
        contents = ocr_blob.download_as_string()
        # load as JSON
        json_object = json.loads(contents)
        # extract text
        full_text = ""
        for response in json_object["responses"]:
            if response["fullTextAnnotation"]:
                full_text += response["fullTextAnnotation"]["text"] + "\r\n"

        # save as a blob
        output_blob_name = f"{ocr_blob.name}.txt"
        import_file_blob = output_bucket.blob(output_blob_name)
        import_file_blob.upload_from_string(full_text)

        # create import file listing
        import_file_entry = {
            "textGcsUri": f"gs://{DESTINATION_BUCKET_NAME}/{output_blob_name}"
        }

        import_file_entries.append(import_file_entry)

print("Extraction completed!")

#### Quick peek at extracted text files

Next, you take a peek at the contents of one of the extracted text files.

In [ ]:
example = import_file_entries[0]["textGcsUri"]

! gsutil cat {example}

### Generate and save import file to be used in `Vertex AI Dataset` resource

You will now build the import file that will be used to create the `Vertex AI Dataset` resource.

In [ ]:
IMPORT_FILE_PATH = "import_file"

# Convert import file entries to JSON Lines format
import_file_content = ""
for entry in import_file_entries:
    import_file_content += json.dumps(entry) + "\n"

print(f"Created import file based on {len(import_file_entries)} annotations")

# Upload content to GCS to be used in our next step
gcs_annotation_file_name = f"{IMPORT_FILE_PATH}/import_file_{TIMESTAMP}.jsonl"
import_file_blob = output_bucket.blob(gcs_annotation_file_name)
import_file_blob.upload_from_string(import_file_content)

print(f"Uploaded import file to {output_bucket.name}/{gcs_annotation_file_name}")

### Create an unlabelled `Vertex AI Dataset` resource

Next, you create the `Dataset` resource using the `create` method for the `TextDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `gcs_source`: A list of one or more dataset index files to import the data items into the `Dataset` resource.
- `import_schema_uri`: The data labeling schema for the data items.

This operation may take ten to twenty minutes.

In [ ]:
print("Creating dataset ...")

dataset = aiplatform.TextDataset.create(
    display_name="Text Dataset " + TIMESTAMP,
    gcs_source=[f"gs://{output_bucket.name}/{gcs_annotation_file_name}"],
    import_schema_uri=DATASET_IMPORT_SCHEMA,
)

print("Completed!")

print(dataset.resource_name)

**Congratulations, your dataset is now ready for annotations!**

You have two options:

*   Use Google Cloud Console to manually annotate the dataset in `Vertex AI`. Checkout [this link](https://cloud.google.com/vertex-ai/docs/datasets/label-using-console#entity-extraction) for more details on how to do so.
*   Create a labelling job to request data labelling. Check out [this link](https://cloud.google.com/vertex-ai/docs/datasets/data-labeling-job) and [this notebook](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage1/get_started_with_data_labeling.ipynb) for more details and examples.


# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.


In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

# Delete the dataset using the Vertex AI fully qualified identifier for the dataset
dataset.delete()

# Delete the bucket created
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI